## TensorBoard

The following is how you do it on Google Colab. 

https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

In other scenarios, after Training the model, run
```tensorboard --logdir=path/to/log-directory```

Tensorboard provides a good visualization tool for all the variables you like and works on a browser.


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Let's fire up tensorboard!

In [0]:
get_ipython().system_raw('tensorboard --logdir ./logs/09_tensorboard --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &') # run nrok to tunnel tensorboard to outside world
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras import initializers
from keras.utils import np_utils
from keras.callbacks import TensorBoard

In [0]:
# Hyper parameters
batch_size = 128
nb_epoch = 100

# Parameters for MNIST dataset
nb_classes = 10

# Parameters for MLP
prob_drop_input = 0.2               # drop probability for dropout @ input layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [0]:
weight_initializer = initializers.RandomNormal(stddev=0.01)

In [0]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_Train = np_utils.to_categorical(y_train, nb_classes)
Y_Test = np_utils.to_categorical(y_test, nb_classes)

In [0]:
# Multilayer Perceptron model
model = Sequential()
model.add(Dense(units=625, input_dim=784, kernel_initializer=weight_initializer, activation='sigmoid', name='dense1'))
model.add(Dropout(prob_drop_input, name='dropout1'))
model.add(Dense(units=625, input_dim=625, kernel_initializer=weight_initializer, activation='sigmoid', name='dense2'))
model.add(Dropout(prob_drop_hidden, name='dropout2'))
model.add(Dense(units=10, input_dim=625, kernel_initializer=weight_initializer, activation='softmax', name='dense3'))
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
# Train
history = model.fit(X_train, Y_Train, validation_data=(X_test, Y_Test), epochs=nb_epoch, batch_size=batch_size, verbose=1,
                    callbacks=[TensorBoard(log_dir='./logs/09_tensorboard', histogram_freq=1)])

In [0]:
# Evaluate
evaluation = model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))